In [1]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

In [6]:
# Reading Data
df_online_retail = pd.read_csv('../../00_Data/data.csv', encoding= 'unicode_escape', na_values='nan')
#df_online_retail_xlsx = pd.read_excel('../00_Data/online_retail.xlsx', sheet_name=0)
#df_online_retail_II = pd.read_excel('../00_Data/online_retail_II.xlsx', sheet_name=0)
#df_rec_sys_order = pd.read_excel('../00_Data/Rec_sys_data.xlsx', sheet_name=0)
#df_rec_sys_customer = pd.read_excel('../00_Data/Rec_sys_data.xlsx', sheet_name=1)
#df_rec_sys_product = pd.read_excel('../00_Data/Rec_sys_data.xlsx', sheet_name=2)

# 1. Online Retail Dataset

## 1.1 Data Split

In [8]:
# Find out number of missing values per column
df_online_retail.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [9]:
# We need to drop missing values in CustomerID
df_online_retail = df_online_retail.dropna(subset=['CustomerID'])

# Sanity check 
df_online_retail.isna().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [10]:
# First check Data Types 
df_online_retail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    406829 non-null  object 
 1   StockCode    406829 non-null  object 
 2   Description  406829 non-null  object 
 3   Quantity     406829 non-null  int64  
 4   InvoiceDate  406829 non-null  object 
 5   UnitPrice    406829 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      406829 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 27.9+ MB


In [11]:
# Since we are going to encode the labels for User and Items, we need to make sure that we convert those to strings 
df_online_retail['CustomerID'] = df_online_retail['CustomerID'].astype(str)

# We can also convert Invoice Date into Date time
df_online_retail['InvoiceDate'] = pd.to_datetime(df_online_retail['InvoiceDate'])
df_online_retail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  object        
 7   Country      406829 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 27.9+ MB


In [12]:
# Last but not least, we will need a column, which indicates that an item was bought by a user. We are assuming, this column is binary - 1 means bought, 0 means no interaction
df_online_retail['purchased'] = 5
df_online_retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,purchased
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,5
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,5
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5


In [13]:
# User & Item  Label Encoder 

# Import LabelEncoder
from sklearn.preprocessing import LabelEncoder

# Set up Objects for User and Item Column
le1 = LabelEncoder()
le2 = LabelEncoder()

# Fit the encoders to the columns in question
le1.fit(df_online_retail['StockCode'])
le2.fit(df_online_retail['CustomerID'])

# Transform dataframe with encoded labels
df_online_retail['StockCode'] = le1.transform(df_online_retail['StockCode'])
df_online_retail['CustomerID'] = le2.transform(df_online_retail['CustomerID'])

# Check Change 
df_online_retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,purchased
0,536365,3249,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,4048,United Kingdom,5
1,536365,2649,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,4048,United Kingdom,5
2,536365,2855,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,4048,United Kingdom,5
3,536365,2803,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,4048,United Kingdom,5
4,536365,2802,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,4048,United Kingdom,5


In [14]:
train, test = train_test_split(df_online_retail, test_size=0.2, random_state=1)

In [15]:
# Print the number of users and items in the training and test sets
print(f'Before Split: {df_online_retail.CustomerID.nunique()} users, {df_online_retail.StockCode.nunique()} items')
print(f'Train: {train.CustomerID.nunique()} users, {train.StockCode.nunique()} items')
print(f'Test: {test.CustomerID.nunique()} users, {test.StockCode.nunique()} items')

Before Split: 4372 users, 3684 items
Train: 4357 users, 3647 items
Test: 4107 users, 3281 items


For an affinity matrix, we need the same number of unique users and items in both datasets. This will be done in the next step

In [16]:
# find the set of unique items and users in both the train and test sets
train_items = set(train['StockCode'].unique())
test_items = set(test['StockCode'].unique())
train_users = set(train['CustomerID'].unique())
test_users = set(test['CustomerID'].unique())

# find the intersection of the sets from step 1
common_items = train_items.intersection(test_items)
common_users = train_users.intersection(test_users)

# filter the train and test sets to include only the rows with item user  IDs that are in the intersection set
train = train[train['StockCode'].isin(common_items)]
train = train[train['CustomerID'].isin(common_users)]
test = test[test['StockCode'].isin(common_items)]
test = test[test['CustomerID'].isin(common_users)]

In [17]:
# Check again, if the number of unique items and users match
print(f'Train: {train.CustomerID.nunique()} users, {train.StockCode.nunique()} items')
print(f'Test: {test.CustomerID.nunique()} users, {test.StockCode.nunique()} items')

Train: 4092 users, 3244 items
Test: 4092 users, 3244 items


In [18]:
print(train.shape)
print(test.shape)

(322853, 9)
(81308, 9)


## 1.2. Create User-Item Matrix

In [40]:
# create a pivot table from the dataframe
train_matrix = pd.pivot_table(train, values='purchased', index='CustomerID', columns='StockCode')

# replace non-zero values with 1 and missing values with 0
train_matrix[train_matrix > 0] = 5
train_matrix = train_matrix.fillna(1)

# Show Matrix
train_matrix.head()

StockCode,0,1,2,3,4,6,7,8,9,10,...,3667,3668,3676,3677,3678,3679,3680,3681,3682,3683
CustomerID,,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,5.0


In [41]:
# create a pivot table from the dataframe
test_matrix = pd.pivot_table(test, values='purchased', index='CustomerID', columns='StockCode', aggfunc=np.sum)

# replace non-zero values with 1 and missing values with 0
test_matrix[test_matrix > 0] = 5
test_matrix = test_matrix.fillna(1)

# Show Matrix
test_matrix.head()

StockCode,0,1,2,3,4,6,7,8,9,10,...,3667,3668,3676,3677,3678,3679,3680,3681,3682,3683
CustomerID,,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0
5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,5.0


In [42]:
# Print Shape of Train & Test Matrix 
print(train_matrix.shape)
print(test_matrix.shape)

(4092, 3244)
(4092, 3244)


In [43]:
# Check any rows that do not only contain 0 as a sanity check
train_matrix.loc[(train_matrix!=0).any(axis=1)]

StockCode,0,1,2,3,4,6,7,8,9,10,...,3667,3668,3676,3677,3678,3679,3680,3681,3682,3683
CustomerID,,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4367,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4368,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4369,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


## 1.3. Model

In [44]:
import logging
import tensorflow as tf


#RBM 
from recommenders.models.rbm.rbm import RBM
from recommenders.utils.timer import Timer
from recommenders.utils.plot import line_graph

# Evaluation
from recommenders.evaluation.python_evaluation import (
    precision_at_k,
    recall_at_k,
    map_at_k,
    auc
)


In [45]:
try:
    del(model)
except:
    pass

In [46]:
#First we initialize the model class
model = RBM(
    possible_ratings=np.setdiff1d(np.unique(train_matrix), np.array([0])),
    visible_units=train_matrix.shape[1],
    hidden_units=50,
    training_epoch=30,
    minibatch_size=600,
    keep_prob=0.7,
    with_metrics=True,
)

In [47]:
#Model Fit
with Timer() as train_time:
    model.fit(train_matrix)

print("Took {:.2f} seconds for training.".format(train_time.interval))

# Plot the train RMSE as a function of the epochs
#line_graph(values=model.rmse_train, labels='train', x_name='epoch', y_name='rmse_train')

Took 9.19 seconds for training.


In [48]:
predicted = model.predict(test_matrix)

In [49]:
predicted

array([[1., 1., 5., ..., 5., 1., 1.],
       [5., 1., 1., ..., 5., 1., 1.],
       [5., 1., 5., ..., 5., 1., 1.],
       ...,
       [1., 5., 1., ..., 5., 1., 1.],
       [1., 1., 5., ..., 5., 1., 1.],
       [1., 1., 5., ..., 5., 1., 1.]], dtype=float32)

In [50]:
np.unique(predicted)

array([1., 5.], dtype=float32)

In [51]:
test_pred = pd.DataFrame(data=predicted, index=test_matrix.index, columns=test_matrix.columns)

In [52]:
test_pred_dfx  = pd.melt(test_pred.reset_index(), id_vars=['CustomerID'], value_vars=test_pred.columns, var_name='StockCode', value_name='predictions')
test_true =  pd.melt(test_matrix.reset_index(), id_vars=['CustomerID'], value_vars=test_matrix.columns, var_name='StockCode', value_name='purchased')

In [62]:
test_true[test_true['purchased']==1]

,CustomerID,StockCode,purchased
0,1,0,1.0
1,2,0,1.0
2,3,0,1.0
3,4,0,1.0
4,5,0,1.0
...,...,...,...
13274443,4367,3683,1.0
13274444,4368,3683,1.0
13274445,4369,3683,1.0
13274446,4370,3683,1.0


In [64]:
test_pred_dfx[test_pred_dfx['predictions']==1]

,CustomerID,StockCode,predictions
0,1,0,1.0
5,7,0,1.0
6,8,0,1.0
7,9,0,1.0
8,10,0,1.0
...,...,...,...
13274443,4367,3683,1.0
13274444,4368,3683,1.0
13274445,4369,3683,1.0
13274446,4370,3683,1.0


In [65]:
test_true.shape

(13274448, 3)

In [66]:
test_pred_dfx.shape

(13274448, 3)

In [67]:
test_pred_dfx.predictions.value_counts()

1.0    9349696
5.0    3924752
Name: predictions, dtype: int64

In [69]:
test_true.purchased.value_counts()

1.0    13203413
5.0       71035
Name: purchased, dtype: int64

In [74]:
# Find out all combinations in test_true that are positive
test_true_positive = test_true[test_true['purchased']==5]
test_true_positive.shape

(71035, 3)

In [75]:
# Merge test_true_positive with test_pred_dfx
test_pred_filtered = pd.merge(test_true_positive[['CustomerID', 'StockCode']], test_pred_dfx, on=['CustomerID', 'StockCode'])
test_pred_filtered.shape

(71035, 3)

## Evaluation

In [55]:
from sklearn.metrics import recall_score, classification_report, confusion_matrix

In [81]:
recall_score_x = recall_score(test_true_positive.purchased, test_pred_filtered.predictions)
print(recall_score_x)

0.0


/Users/arasdirekoglu/opt/anaconda3/envs/recom/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [85]:
print(classification_report(test_true_positive.purchased, test_pred_filtered.predictions))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         0
         5.0       1.00      0.41      0.58     71035

    accuracy                           0.41     71035
   macro avg       0.50      0.20      0.29     71035
weighted avg       1.00      0.41      0.58     71035



/Users/arasdirekoglu/opt/anaconda3/envs/recom/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/arasdirekoglu/opt/anaconda3/envs/recom/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/arasdirekoglu/opt/anaconda3/envs/recom/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

In [87]:
print(confusion_matrix(test_true_positive.purchased, test_pred_filtered.predictions))

[[    0     0]
 [42053 28982]]


In [91]:
true_positive = confusion_matrix(test_true_positive.purchased, test_pred_filtered.predictions)[1][1] 
false_negative = confusion_matrix(test_true_positive.purchased, test_pred_filtered.predictions)[1][0]
combined = true_positive + false_negative
recall = true_positive / combined
print(recall) 

0.407996058281129


In [106]:
precision = test_pred_filtered.predictions.value_counts()[5] / test_pred_filtered.predictions.value_counts()[1]
print(precision)

0.6891779421206573
